In [1]:
import keras as k
import scipy
import numpy as np
import pandas as pd
import pickle

Using Theano backend.


In [2]:
def get_scaler_openfoam(home,file,fsize):
    file = file + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    data = pd.read_csv(home+file,skiprows=21,skipfooter=33)
    data = data.values
    return data
    


In [3]:
def get_vector_openfoam(home,file,fsize):
    file = file + str(fsize) + 'X' + str(fsize) + 'X' + str(fsize)
    data = pd.read_csv(home+file,skiprows=21,skipfooter=33)
    data = data.values
    data = np.array([[float(x[0].split()[0][1:]), float(x[0].split()[1]), float(x[0].split()[2][:-1])] for x in data])
    return data

In [4]:
def dumpfiles(home, fsize, timestep):
    home = home + '/' + str(timestep) + '/'
    numpy_barAlpp = get_scaler_openfoam(home,'barAlpp',fsize)
    numpy_tildeUs = get_vector_openfoam(home,'tildeUs',fsize)
    numpy_tildeUf = get_vector_openfoam(home,'tildeUf',fsize)
    numpy_barAlppVdrift = get_vector_openfoam(home,'barAlppVdrift',fsize)
    numpy_barGradPg = get_vector_openfoam(home,'barGradPg',fsize)
    numpy_Xsgs = get_vector_openfoam(home,'Xsgs',fsize)
    #numpy_Ysgs = get_vector_openfoam(home,'Ysgs',fsize)
    
    numpy_Uslip = numpy_tildeUf - numpy_tildeUs

    original_table = np.concatenate((numpy_barAlpp,numpy_Uslip,numpy_barAlppVdrift,numpy_barGradPg ),axis=1)

    nx = 80
    ny = 80
    nz = 320
    dx = 1

    index1=0
    npData=original_table.shape[-1]

    myData = np.zeros( ((nx-2)*(ny-2)*(nz-2), npData * (2*dx +1)**3) );
    target = np.zeros((nx-2)*(ny-2)*(nz-2))
    for i in range(1,nx-1):
        for j in range(1,ny-1):
            for k in range(1,nz-1):


                index2 = 0;
                for ii in range(-dx,dx+1):
                    for jj in range(-dx,dx+1):
                        for kk in range(-dx,dx+1):
                            stencil= i+ii + (j+jj)*nx + (k+kk)*nx*ny;
                            target[index1] = numpy_Xsgs[stencil,-1]
                            temp = original_table[stencil][:]
                            for ind in range(npData):
                                myData[index1,index2] = temp[ind];
                                index2 = index2 + 1
                index1 = index1 +1   
    myData = np.hstack((np.ones((myData.shape[0],1))*(1/(3*fsize)),myData))
    file_myData = "myData_deltap_vdrift"+'fsize' + str(fsize)+'timestep'+str(timestep)+".pickle"
    file_target = "target_deltap_vdrift"+'fsize' + str(fsize)+'timestep'+str(timestep)+".pickle"
    pickle.dump( myData, open( file_myData, "wb" ) )
    pickle.dump( target, open( file_target, "wb" ) )
    return myData

In [5]:
home = '/home/nandhini/yundij/neural_network'

In [6]:
dumpfiles(home=home, fsize=9, timestep=0.3)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


array([[  3.70370370e-02,   2.11182000e-02,   2.27890000e-01, ...,
          1.66577000e+02,  -9.29904000e+02,   1.99968000e+03],
       [  3.70370370e-02,   2.14844000e-02,   2.10855000e-01, ...,
          2.02942000e+02,  -9.58469000e+02,   1.97586000e+03],
       [  3.70370370e-02,   2.20235000e-02,   1.87866000e-01, ...,
          2.22511000e+02,  -9.83956000e+02,   1.88230000e+03],
       ..., 
       [  3.70370370e-02,   1.80045000e-02,   4.07704000e-01, ...,
         -2.22847000e+02,  -1.07062000e+03,   1.77781000e+03],
       [  3.70370370e-02,   1.81380000e-02,   4.00299000e-01, ...,
         -1.74034000e+02,  -1.10248000e+03,   1.75963000e+03],
       [  3.70370370e-02,   1.86405000e-02,   3.79785000e-01, ...,
         -1.19770000e+02,  -1.12295000e+03,   1.77706000e+03]])

In [7]:
dumpfiles(home=home, fsize=9, timestep=0.4)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


array([[  3.70370370e-02,   2.44372000e-02,   1.38228260e-01, ...,
         -1.96361000e+03,   1.19009000e+03,  -1.06702000e+03],
       [  3.70370370e-02,   3.12508000e-02,   1.22493900e-01, ...,
         -2.11797000e+03,   1.23790000e+03,  -1.16226000e+03],
       [  3.70370370e-02,   3.81400000e-02,   1.00329700e-01, ...,
         -2.17406000e+03,   1.23823000e+03,  -1.33254000e+03],
       ..., 
       [  3.70370370e-02,   5.85147000e-03,   2.00954400e-01, ...,
         -5.60311000e+02,   9.18948000e+02,   1.10672000e+03],
       [  3.70370370e-02,   6.35617000e-03,   1.65289800e-01, ...,
         -7.19528000e+02,   9.84287000e+02,   7.84686000e+02],
       [  3.70370370e-02,   7.28664000e-03,   1.35843100e-01, ...,
         -9.50027000e+02,   1.07213000e+03,   3.34588000e+02]])

In [8]:
dumpfiles(home=home, fsize=9, timestep=0.35)

/home/hema/yile/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  app.launch_new_instance()


array([[  3.70370370e-02,   1.53036000e-01,  -1.50170000e-02, ...,
          1.12544000e+03,   5.94233000e+02,   5.01976000e+02],
       [  3.70370370e-02,   1.42752000e-01,  -1.48680000e-02, ...,
          1.12349000e+03,   5.13485000e+02,   6.27682000e+02],
       [  3.70370370e-02,   1.41167000e-01,  -1.07470000e-02, ...,
          1.07053000e+03,   3.07105000e+02,   5.61029000e+02],
       ..., 
       [  3.70370370e-02,   1.99601000e-01,  -2.70020000e-02, ...,
          1.19130000e+03,  -1.96207000e+03,  -3.36464000e+03],
       [  3.70370370e-02,   1.87785000e-01,  -3.04880000e-02, ...,
          1.24554000e+03,  -1.45336000e+03,  -2.80335000e+03],
       [  3.70370370e-02,   1.72717000e-01,  -4.08150000e-02, ...,
          1.38840000e+03,  -1.01943000e+03,  -2.16498000e+03]])

In [ ]:
dumpfiles(home=home, dp = 300, timestep=0.6, fsize=9)

In [ ]:
dumpfiles(home=home, dp = 150, timestep=0.55, fsize=11)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.65, fsize=13)

In [ ]:
dumpfiles(home=home, dp = 75, timestep=0.325, fsize=13,phi=0.3, cohesion='HighCoh')